In [ ]:
# Install necessary libraries
!pip install torch torchvision pandas scikit-learn pillow

# Import libraries
import os
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split

# Define constants
IMG_HEIGHT = 224
IMG_WIDTH = 224
NUM_CLASSES = 49
BATCH_SIZE = 32
EPOCHS = 100
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define paths
drive_base_path = '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR1/VRL_challenge_PAR/'
train_path = os.path.join(drive_base_path, 'train.txt')
images_folder = os.path.join(drive_base_path, 'images')

# Load dataset
train_df = pd.read_csv(train_path, sep=' ', header=None)
image_names = train_df.iloc[:, 0].astype(str).values
labels = train_df.iloc[:, 1:].values.astype(int)

# Split dataset
image_names_train, image_names_val, labels_train, labels_val = train_test_split(image_names, labels, test_size=0.2, random_state=42)

# Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, image_names, labels, images_folder, transform=None):
        self.image_names = image_names
        self.labels = labels
        self.images_folder = images_folder
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        img_path = os.path.join(self.images_folder, f"{img_name}.jpg")
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

# Define transforms
transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create DataLoader
train_dataset = CustomDataset(image_names_train, labels_train, images_folder, transform=transform)
val_dataset = CustomDataset(image_names_val, labels_val, images_folder, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Define the DenseNet model
class PretrainedDenseNet(nn.Module):
    def __init__(self, num_classes):
        super(PretrainedDenseNet, self).__init__()
        self.densenet = models.densenet121(pretrained=True)
        num_ftrs = self.densenet.classifier.in_features
        self.densenet.classifier = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.densenet(x)

# Define the SimpleCNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Instantiate the models
densenet_model = PretrainedDenseNet(NUM_CLASSES).to(DEVICE)
simplecnn_model = SimpleCNN(NUM_CLASSES).to(DEVICE)

# Define the ensemble model
class EnsembleModel(nn.Module):
    def __init__(self, densenet_model, simplecnn_model):
        super(EnsembleModel, self).__init__()
        self.densenet_model = densenet_model
        self.simplecnn_model = simplecnn_model
        self.classifier = nn.Sequential(
            nn.Linear(NUM_CLASSES * 2, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, NUM_CLASSES),
            nn.Sigmoid()
        )

    def forward(self, x):
        densenet_output = self.densenet_model(x)
        simplecnn_output = self.simplecnn_model(x)
        combined_output = torch.cat((densenet_output, simplecnn_output), dim=1)
        return self.classifier(combined_output)

# Instantiate the ensemble model
ensemble_model = EnsembleModel(densenet_model, simplecnn_model).to(DEVICE)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(ensemble_model.parameters(), lr=0.001)

# Training loop
for epoch in range(EPOCHS):
    ensemble_model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE).float()
        optimizer.zero_grad()
        outputs = ensemble_model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {epoch_loss:.4f}')

# Save the trained model
model_path = "ensemble_model.pth"
torch.save(ensemble_model.state_dict(), model_path)
print(f"Model saved to {model_path}")

# Validation
# Validation
ensemble_model.eval()
val_loss = 0.0
val_corrects = 0
with torch.no_grad():
    for inputs, targets in val_loader:
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE).float()
        outputs = ensemble_model(inputs)
        loss = criterion(outputs, targets)
        val_loss += loss.item() * inputs.size(0)

        # Binary prediction for multi-label classification
        preds = (outputs > 0.5).float()

        # Compute correct predictions
        val_corrects += (preds == targets).sum().item()

# Average loss over validation set
val_loss = val_loss / len(val_loader.dataset)

# Average accuracy per sample
val_accuracy = val_corrects / (len(val_loader.dataset) * NUM_CLASSES)

print(f'Validation Loss: {val_loss:.4f}')
print(f'Validation Accuracy: {val_accuracy:.4f}')



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 119MB/s]


Epoch 1/100, Loss: 0.4926
Epoch 2/100, Loss: 0.4026
Epoch 3/100, Loss: 0.3922
Epoch 4/100, Loss: 0.3873
Epoch 5/100, Loss: 0.3821
Epoch 6/100, Loss: 0.3748
Epoch 7/100, Loss: 0.3720
Epoch 8/100, Loss: 0.3640
Epoch 9/100, Loss: 0.3614
Epoch 10/100, Loss: 0.3576
Epoch 11/100, Loss: 0.3528
Epoch 12/100, Loss: 0.3572
Epoch 13/100, Loss: 0.3541
Epoch 14/100, Loss: 0.3511
Epoch 15/100, Loss: 0.3521
Epoch 16/100, Loss: 0.3488
Epoch 17/100, Loss: 0.3491
Epoch 18/100, Loss: 0.3447
Epoch 19/100, Loss: 0.3433
Epoch 20/100, Loss: 0.3413
Epoch 21/100, Loss: 0.3399
Epoch 22/100, Loss: 0.3416
Epoch 23/100, Loss: 0.3410
Epoch 24/100, Loss: 0.3377
Epoch 25/100, Loss: 0.3392
Epoch 26/100, Loss: 0.3444
Epoch 27/100, Loss: 0.3401
Epoch 28/100, Loss: 0.3392
Epoch 29/100, Loss: 0.3335
Epoch 30/100, Loss: 0.3367
Epoch 31/100, Loss: 0.3338
Epoch 32/100, Loss: 0.3317
Epoch 33/100, Loss: 0.3341
Epoch 34/100, Loss: 0.3294
Epoch 35/100, Loss: 0.3353
Epoch 36/100, Loss: 0.3338
Epoch 37/100, Loss: 0.3263
Epoch 38/1

In [ ]:
# Install necessary libraries
!pip install torch torchvision pandas scikit-learn pillow

# Import libraries
import os
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split

# Define constants
IMG_HEIGHT = 224
IMG_WIDTH = 224
NUM_CLASSES = 49
BATCH_SIZE = 32
EPOCHS = 500
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define paths
drive_base_path = '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR1/VRL_challenge_PAR/'
train_path = os.path.join(drive_base_path, 'train.txt')
images_folder = os.path.join(drive_base_path, 'images')

# Load dataset
train_df = pd.read_csv(train_path, sep=' ', header=None)
image_names = train_df.iloc[:, 0].astype(str).values
labels = train_df.iloc[:, 1:].values.astype(int)

# Split dataset
image_names_train, image_names_val, labels_train, labels_val = train_test_split(image_names, labels, test_size=0.2, random_state=42)

# Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, image_names, labels, images_folder, transform=None):
        self.image_names = image_names
        self.labels = labels
        self.images_folder = images_folder
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        img_path = os.path.join(self.images_folder, f"{img_name}.jpg")
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

# Define transforms
transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create DataLoader
train_dataset = CustomDataset(image_names_train, labels_train, images_folder, transform=transform)
val_dataset = CustomDataset(image_names_val, labels_val, images_folder, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Define the DenseNet model
class PretrainedDenseNet(nn.Module):
    def __init__(self, num_classes):
        super(PretrainedDenseNet, self).__init__()
        self.densenet = models.densenet121(pretrained=True)
        num_ftrs = self.densenet.classifier.in_features
        self.densenet.classifier = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.densenet(x)

# Define the SimpleCNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Instantiate the models
densenet_model = PretrainedDenseNet(NUM_CLASSES).to(DEVICE)
simplecnn_model = SimpleCNN(NUM_CLASSES).to(DEVICE)

# Define the ensemble model
class EnsembleModel(nn.Module):
    def __init__(self, densenet_model, simplecnn_model):
        super(EnsembleModel, self).__init__()
        self.densenet_model = densenet_model
        self.simplecnn_model = simplecnn_model
        self.classifier = nn.Sequential(
            nn.Linear(NUM_CLASSES * 2, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, NUM_CLASSES),
            nn.Sigmoid()
        )

    def forward(self, x):
        densenet_output = self.densenet_model(x)
        simplecnn_output = self.simplecnn_model(x)
        combined_output = torch.cat((densenet_output, simplecnn_output), dim=1)
        return self.classifier(combined_output)

# Instantiate the ensemble model
ensemble_model = EnsembleModel(densenet_model, simplecnn_model).to(DEVICE)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(ensemble_model.parameters(), lr=0.001)

# Training loop
for epoch in range(EPOCHS):
    ensemble_model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE).float()
        optimizer.zero_grad()
        outputs = ensemble_model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)

    # Validation
    ensemble_model.eval()
    val_loss = 0.0
    val_corrects = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE).float()
            outputs = ensemble_model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)

            # Binary prediction for multi-label classification
            preds = (outputs > 0.5).float()

            # Compute correct predictions
            val_corrects += (preds == targets).sum().item()

    val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = val_corrects / (len(val_loader.dataset) * NUM_CLASSES)

    print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {epoch_loss:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

# Save the trained model
model_path = "ensemble_model.pth"
torch.save(ensemble_model.state_dict(), model_path)
print(f"Model saved to {model_path}")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/500, Loss: 0.4960, Validation Loss: 0.4050, Validation Accuracy: 0.8350
Epoch 2/500, Loss: 0.3998, Validation Loss: 0.3869, Validation Accuracy: 0.8354
Epoch 3/500, Loss: 0.3903, Validation Loss: 0.3876, Validation Accuracy: 0.8354
Epoch 4/500, Loss: 0.3866, Validation Loss: 0.3795, Validation Accuracy: 0.8391
Epoch 5/500, Loss: 0.3782, Validation Loss: 0.3745, Validation Accuracy: 0.8406
Epoch 6/500, Loss: 0.3682, Validation Loss: 0.3860, Validation Accuracy: 0.8354
Epoch 7/500, Loss: 0.3667, Validation Loss: 0.3790, Validation Accuracy: 0.8354
Epoch 8/500, Loss: 0.3576, Validation Loss: 0.3856, Validation Accuracy: 0.8372
Epoch 9/500, Loss: 0.3555, Validation Loss: 0.3753, Validation Accuracy: 0.8437
Epoch 10/500, Loss: 0.3568, Validation Loss: 0.3658, Validation Accuracy: 0.8442
Epoch 11/500, Loss: 0.3516, Validation Loss: 0.3531, Validation Accuracy: 0.8539
Epoch 12/500, Loss: 0.3496, Validation Loss: 0.3627, Validation Accuracy: 0.8500
Epoch 13/500, Loss: 0.3477, Validatio

In [ ]:
!pip install torch torchvision


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
!pip install --upgrade pip
!pip install --upgrade setuptools


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Using cached setuptools-70.0.0-py3-none-any.whl.metadata (5.9 kB)
Using cached setuptools-70.0.0-py3-none-any.whl (863 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# Define constants
IMG_HEIGHT = 224
IMG_WIDTH = 224
NUM_CLASSES = 49
BATCH_SIZE = 32
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define paths
drive_base_path = '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR1/VRL_challenge_PAR/'
test_images_folder = os.path.join(drive_base_path, 'test_images')  # Folder containing test images

# List all test image files
test_image_names = [f.split('.')[0] for f in os.listdir(test_images_folder) if f.endswith('.jpg')]

# Custom Dataset for Test Data
class TestDataset(Dataset):
    def __init__(self, image_names, images_folder, transform=None):
        self.image_names = image_names
        self.images_folder = images_folder
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        img_path = os.path.join(self.images_folder, f"{img_name}.jpg")
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, img_name

# Define transforms
transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create DataLoader for test data
test_dataset = TestDataset(test_image_names, test_images_folder, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Define the EnsembleModel (same as used in training)
class PretrainedDenseNet(nn.Module):
    def __init__(self, num_classes):
        super(PretrainedDenseNet, self).__init__()
        self.densenet = models.densenet121(pretrained=True)
        num_ftrs = self.densenet.classifier.in_features
        self.densenet.classifier = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.densenet(x)

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

class EnsembleModel(nn.Module):
    def __init__(self, densenet_model, simplecnn_model):
        super(EnsembleModel, self).__init__()
        self.densenet_model = densenet_model
        self.simplecnn_model = simplecnn_model
        self.classifier = nn.Sequential(
            nn.Linear(NUM_CLASSES * 2, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, NUM_CLASSES),
            nn.Sigmoid()
        )

    def forward(self, x):
        densenet_output = self.densenet_model(x)
        simplecnn_output = self.simplecnn_model(x)
        combined_output = torch.cat((densenet_output, simplecnn_output), dim=1)
        return self.classifier(combined_output)

# Instantiate the models
densenet_model = PretrainedDenseNet(NUM_CLASSES).to(DEVICE)
simplecnn_model = SimpleCNN(NUM_CLASSES).to(DEVICE)

# Instantiate the ensemble model
ensemble_model = EnsembleModel(densenet_model, simplecnn_model).to(DEVICE)

# Load the trained ensemble model
model_path = "/content/ensemble_model.pth"
ensemble_model.load_state_dict(torch.load(model_path))
ensemble_model.eval()

# Function to make predictions on test data
def predict(model, dataloader, threshold=0.5):
    model.eval()
    predictions = {}
    with torch.no_grad():
        for inputs, img_names in dataloader:
            inputs = inputs.to(DEVICE)
            outputs = model(inputs)
            outputs = outputs.cpu().numpy()
            binary_outputs = (outputs >= threshold).astype(int)
            for img_name, output in zip(img_names, binary_outputs):
                predictions[img_name] = output
    return predictions

# Make predictions on the test dataset

# Make predictions on the test dataset
predictions = predict(model, test_loader)

# Example: Print the predictions for the first few test images
for img_name, output in list(predictions.items())[:5]:
    print(f"Image: {img_name}, Prediction: {output}")

# Save predictions to a CSV file
predictions_df = pd.DataFrame.from_dict(predictions, orient='index')
predictions_df.to_csv('binary_predictions.csv', header=False)

Image: 144, Prediction: [1 0 0 0 0 0 1 0 0 1 1 0 1 1 1 1 1 0 0 1 0 0 0 1 0 1 1 0 1 1 1 0 0 1 0 1 0
 0 1 0 1 1 0 1 0 1 0 0 0]
Image: 24, Prediction: [1 0 0 0 0 0 1 0 1 1 1 0 1 1 1 0 1 0 0 1 0 0 0 1 0 1 0 0 1 1 1 0 0 1 0 1 0
 0 0 0 1 1 0 1 0 1 0 0 0]
Image: 124, Prediction: [1 0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 0 0 1 0 0 0 1 0 1 1 0 1 1 1 0 0 1 0 1 0
 0 0 0 1 1 0 1 0 0 1 0 0]
Image: 66, Prediction: [1 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 0 0 0 1 0 0 0 1 0 1 1 0 1 1 1 1 0 1 0 1 0
 0 0 0 1 1 0 1 1 0 0 0 0]
Image: 107, Prediction: [1 0 0 0 1 1 1 0 1 1 1 0 0 1 1 0 0 0 0 1 0 0 0 1 0 1 1 0 1 1 1 0 0 1 0 1 0
 0 0 0 1 1 0 1 1 0 0 0 0]
